# Exercises - June 7 2019

### Day 13
### June 7 2019 
### P. Sudarshan Kumar

## Mission 1: Cross Validation Introduction

### Instructions

        ● Use the numpy.random.permutation() function to shu􀆨e the ordering of the rows in
        dc_listings.
        
        ● Select the first 1862 rows and assign to split_one.
        
        ● Select the remaining 1861 rows and assign to split_two.

In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
def read(filename):
    df = pd.read_csv(filename)
    return df
df_str = read('Data Files/dc_listings.csv')
rand = np.random.permutation(df_str.index)
df = df.loc[rand]
split_one = df.iloc[0:1862]
split_two = df.iloc[1862:]

NameError: name 'df' is not defined

## Mission 2: Holdout Validation

### Instructions

        ● Train a k-nearest neighbors model using the default algorithm (auto) and the default
        number of neighbors (5) that:
        
            ○ Uses the accommodates column from train_one for training and

            ○ Tests it on test_one.

        ● Assign the resulting RMSE value to iteration_one_rmse.
        
        ● Train a k-nearest neighbors model using the default algorithm (auto) and the default
        number of neighbors (5) that:
        
            ○ Uses the accommodates column from train_two for training and

            ○ Tests it on test_two.
        
        ● Assign the resulting RMSE value to iteration_two_rmse.
        
        ● Use numpy.mean() to calculate the average of the 2 RMSE values and assign to
        avg_rmse.

In [ ]:
knn = KNeighborsRegressor(algorithm='brute')
train_features = split_one[['accommodates']]
train_target = split_one['price']
knn.fit(train_features, train_target)
predictions = knn.predict(split_two[['accommodates']])
iteration_one_rmse = np.sqrt(mean_squared_error(split_two['price'], predictions))
iteration_one_rmse

In [ ]:
knn = KNeighborsRegressor(algorithm='brute')
train_features = split_two[['accommodates']]
train_target = split_two['price']
knn.fit(train_features, train_target)
predictions = knn.predict(split_one[['accommodates']])
iteration_two_rmse = np.sqrt(mean_squared_error(split_one['price'], predictions))
iteration_two_rmse

In [ ]:
avg_rmse = np.mean([iteration_one_rmse,iteration_two_rmse])

## Mission 3: K-Fold Cross Validation
### Instructions
    ● Add a new column to dc_listings named fold that contains the fold number each row
    belongs to:
    
        ○ Fold 1 should have rows from index 0 up to745, not including 745.

        ○ Fold 2 should have rows from index 745 up to 1490, not including 1490.

        ○ Fold 3 should have rows from index 1490 up to 2234, not including 2234.

        ○ Fold 4 should have rows from index 2234 up to 2978, not including 2978.

        ○ Fold 5 should have rows from index 2978 up to 3723, not including 3723.

    ● Display the unique value counts for the fold column to confirm that each fold has
    roughly the same number of elements.
    
    ● Display the number of missing values in the fold column to confirm we didn't miss any
    rows.

In [6]:
pd.options.mode.chained_assignment = None
df_str['Fold'] = 0
df_str['Fold'][:745] = 1
df_str['Fold'][745:1490] = 2
df_str['Fold'][1490:2234] = 3
df_str['Fold'][2234:2978] = 4
df_str['Fold'][2978:3723] = 5
df_str['Fold'].value_counts()

5    745
1    745
2    745
3    744
4    744
Name: Fold, dtype: int64

In [7]:
df_str['Fold'].isnull().sum(axis=0)

0

## Mission 4: First Iteration
### Instructions
    ● Train a k-nearest neighbors model using the accommodates column as the sole feature
    from folds 2 to 5 as the training set.
    
    ● Use the model to make predictions on the test set (accommodates column from fold 1)
    and assign the predicted labels to labels.
    
    ● Calculate the RMSE value by comparing the price column with the predicted labels.
    
    ● Assign the RMSE value to iteration_one_rmse.

In [9]:
model = KNeighborsRegressor()
train_iteration_one = df_str[df_str["Fold"] != 1]
test_iteration_one = df_str[df_str["Fold"] == 1].copy()
model.fit(train_iteration_one[["accommodates"]], train_iteration_one["price"])
test_iteration_one["predicted_price"] = model.predict(test_iteration_one[["accommodates"]])
iteration_one_mse = mean_squared_error(test_iteration_one["price"],test_iteration_one["predicted_price"])
iteration_one_rmse = iteration_one_mse ** (1/2)
iteration_one_rmse

110.30276673317532

## Mission 5: Function for Training Models
### Instructions
    ● Write a function named train_and_validate that takes in a dataframe as the first
    parameter (df) and a list of fold values (1 to 5 in our case) as the second parameter
    (folds). This function should:
    
        ○ Train n models (where n is number of folds) and perform k-fold cross validation
        (using n folds). Use the default k value for the KNeighborsRegressor class.

        ○ Return a list of RMSE values, where the first element is the RMSE for when fold
        1 was the test set, the second element is the RMSE for when fold 2 was the test
        set, and so on.

    ● Use the train_and_validate function to return the list of RMSE values for the
    dc_listings Dataframe and assign to rmses.
    
    ● Calculate the mean of these values and assign to avg_rmse.
    
    ● Display both rmses and avg_rmse.

In [21]:
def train_and_validate(df_str,folds):
    rmses = []
    for i in range (1,folds+1):
        model = KNeighborsRegressor()
        train_iteration_one = df_str[df_str["Fold"] != i]
        test_iteration_one = df_str[df_str["Fold"] == i].copy()
        model.fit(train_iteration_one[["accommodates"]], train_iteration_one["price"])
        test_iteration_one["predicted_price"] = model.predict(test_iteration_one[["accommodates"]])
        iteration_one_mse = mean_squared_error(test_iteration_one["price"],test_iteration_one["predicted_price"])
        rmses.append(iteration_one_mse ** (1/2))
    return rmses
rmses = train_and_validate(df_str,5)
avg_rmse = np.mean(rmses)
print(rmses,'\n',avg_rmse)

[110.30276673317532, 78.91359168749155, 71.45196860272152, 96.25215121182086, 243.6917631686422] 
 120.12244828077027


## Mission 6: Performing K-Fold Cross Validation Using Scikit-Learn
### Instructions
    ● Create a new instance of the KFold class with the following properties:
    
        ○ 5 folds,

        ○ shuffle set to True,

        ○ random seed set to 1 (so we can answer check using the same seed),

        ○ assigned to the variable kf.

    ● Create a new instance of the KNeighborsRegressor class and assign to knn.
    
    ● Use the cross_val_score() function to perform k-fold cross-validation:
    
        ○ using the KNeighborsRegressor instance knn,

        ○ using the accommodates column for training,

        ○ using the price column as the target column,

        ○ returning an array of MSE values (one value for each fold).

    ● Assign the resulting list of MSE values to mses. Then, take the absolute value followed
    by the square root of each MSE value. Then, calculate the average of the resulting RMSE
    values and assign to avg_rmse.

In [29]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kf = KFold(5, shuffle=True, random_state=1)
knn = KNeighborsRegressor()
mses = cross_val_score(knn, df_str[['accommodates']],df_str['price'],scoring=None, cv=kf)
np.mean(mses)

0.16370696694676756